In [1]:
import numpy as np
import pandas as pd

import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress 
from scipy import stats
import scipy.stats as sts
# import hvplot.pandas

import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# Import data
combined_news_df = pd.read_csv('../Data/Alicia/Combined-NewsData-CSV.csv')
combined_news_df.head()


,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies
0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98 Comments,6 Shares,1 Reply
1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98 Comments,31 Shares,1 Reply
2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,1.2K Comments,119 Shares,112 Replies
3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202 Comments,37 Shares,9 Replies
4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390 Comments,83 Shares,View more comments


In [3]:
# Split the values to show only numeric without the words like Shares, Comments, Replies
combined_news_df["Facebook_likes"] = combined_news_df["Facebook_likes"].str.split(" ", n = 1, expand = True)
combined_news_df["Facebook_comments"] = combined_news_df["Facebook_comments"].str.split(" ", n = 1, expand = True)
combined_news_df["Facebook_shares"] = combined_news_df["Facebook_shares"].str.split(" ", n = 1, expand = True)
combined_news_df["Comment_replies"] = combined_news_df["Comment_replies"].str.split(" ", n = 1, expand = True)

In [4]:
combined_news_df

,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies
0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98,6,1
1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98,31,1
2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,1.2K,119,112
3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202,37,9
4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390,83,View
...,...,...,...,...,...,...,...,...,...
1040,Fox News,BREAKING: The IRS may have decided to heed the...,IRS planning to delay tax filing deadline unti...,https://www.facebook.com/FoxNews/?__cft__[0]=A...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,588,314,View
1041,Fox News was live.,White House press secretary Jen Psaki holds a ...,NaN,https://www.facebook.com/FoxNews/?__cft__[0]=A...,NaN,5.8K,12K,461,View
1042,Fox News was live.,Federal Reserve Chairman Jerome Powell holds a...,NaN,https://www.facebook.com/FoxNews/?__cft__[0]=A...,NaN,1.6K,2.9K,275,7
1043,Fox News,NaN,"Fed holds rates near zero, lifts growth outloo...",https://www.facebook.com/FoxNews/?__cft__[0]=A...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,890,619,91,View


In [5]:
# To replace the "view" text from the column "Comment_replies" with zero 
combined_news_df["Comment_replies"] = combined_news_df["Comment_replies"].replace(to_replace ="View", value ="0")

In [6]:
combined_news_df.head()

,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies
0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98,6,1
1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98,31,1
2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,1.2K,119,112
3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202,37,9
4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390,83,0


In [7]:
# Convert the "K" to thousands in numerical 
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000
    return x


In [8]:
combined_news_df["Facebook_comments"] = combined_news_df["Facebook_comments"].apply(value_to_float)
combined_news_df["Facebook_likes"] = combined_news_df["Facebook_likes"].apply(value_to_float)
combined_news_df["Facebook_shares"] = combined_news_df["Facebook_shares"].apply(value_to_float)


In [9]:
combined_news_df

,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies
0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98,6,1
1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98,31,1
2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1100.0,1200.0,119,112
3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202,37,9
4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390,83,0
...,...,...,...,...,...,...,...,...,...
1040,Fox News,BREAKING: The IRS may have decided to heed the...,IRS planning to delay tax filing deadline unti...,https://www.facebook.com/FoxNews/?__cft__[0]=A...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1100.0,588,314,0
1041,Fox News was live.,White House press secretary Jen Psaki holds a ...,NaN,https://www.facebook.com/FoxNews/?__cft__[0]=A...,NaN,5800.0,12000.0,461,0
1042,Fox News was live.,Federal Reserve Chairman Jerome Powell holds a...,NaN,https://www.facebook.com/FoxNews/?__cft__[0]=A...,NaN,1600.0,2900.0,275,7
1043,Fox News,NaN,"Fed holds rates near zero, lifts growth outloo...",https://www.facebook.com/FoxNews/?__cft__[0]=A...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,890,619,91,0


In [10]:
# Convert to new csv file
cleaned_df = combined_news_df.to_csv("cleaned_data")
cleaned_df
